In [24]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from skimage.feature import hog
import matplotlib.image as mpimg
import glob
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
%matplotlib inline

In [22]:
class PreProcessing:
    
    def __init__(self, X_scaler=None, nbins=32, 
                 orient=8, pix_per_cell=8, cell_per_block=2, 
                 conv='RGB2YCrCb', size=(32,32)):
        self.conv = conv
        self.size = size
        self.orient = orient
        self.pix_per_cell = pix_per_cell
        self.cell_per_block = cell_per_block
        self.nbins = nbins
        self.X_scaler = X_scaler
        
        
    def set_image(self, image_file):
        image = mpimg.imread(image_file)
        image = image.astype(np.float32)/255
        if self.conv != None:
            self.image = self.convert_color(image, self.conv)
        else:
            self.image = image
        self.ch1 = self.image[:,:,0]
        self.ch2 = self.image[:,:,1]
        self.ch3 = self.image[:,:,2]
        
        
    def convert_color(self, img, conv='RGB2YCrCb'):
        if conv == 'RGB2YCrCb':
            return cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
        if conv == 'BGR2YCrCb':
            return cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
        if conv == 'RGB2LUV':
            return cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
        
        
    def get_bin_spatial(self):
        color1 = cv2.resize(self.ch1, self.size).ravel()
        color2 = cv2.resize(self.ch2, self.size).ravel()
        color3 = cv2.resize(self.ch3, self.size).ravel()
        return np.hstack((color1, color2, color3))
                
        
    def get_color_hist(self):    #bins_range=(0, 256)
        # Compute the histogram of the color channels separately
        channel1_hist = np.histogram(self.ch1, bins=self.nbins)
        channel2_hist = np.histogram(self.ch2, bins=self.nbins)
        channel3_hist = np.histogram(self.ch3, bins=self.nbins)
        # Concatenate the histograms into a single feature vector
        hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
        # Return the individual histograms, bin_centers and feature vector
        return hist_features
    
    
    def get_hog_features(self, image, vis=False, feature_vec=True):
        if vis == True:
            features, hog_image = hog(image, orientations=self.orient, 
                                      pixels_per_cell=(self.pix_per_cell, self.pix_per_cell),
                                      cells_per_block=(self.cell_per_block, self.cell_per_block), 
                                      transform_sqrt=False, 
                                      visualise=vis, feature_vector=feature_vec)
            return features, hog_image
        else:   
            features = hog(image, orientations=self.orient, 
                           pixels_per_cell=(self.pix_per_cell, self.pix_per_cell),
                           cells_per_block=(self.cell_per_block, self.cell_per_block), 
                           transform_sqrt=False, 
                           visualise=vis, feature_vector=feature_vec)
            return features 
        
        
    def get_all_hog_features(self, vis=False, feature_vec=True):
        if vis == True:            
            features_1, hog_image_1 = self.get_hog_features(self.ch1, vis, feature_vec)
            features_2, hog_image_2 = self.get_hog_features(self.ch2, vis, feature_vec)
            features_3, hog_image_3 = self.get_hog_features(self.ch3, vis, feature_vec)
            return np.hstack((features_1, features_2, features_3)), np.array([hog_image_1, hog_image_2, hog_image_3])
        else:      
            features_1 = self.get_hog_features(self.ch1, vis, feature_vec)
            features_2 = self.get_hog_features(self.ch2, vis, feature_vec)
            features_3 = self.get_hog_features(self.ch3, vis, feature_vec)
            return np.hstack((features_1, features_2, features_3))
        
        
    def get_features_for_image(self, vis=False, feature_vec=True):
        hog_features = self.get_all_hog_features(vis, feature_vec)
        spatial_features = self.get_bin_spatial()
        hist_features = self.get_color_hist()
        
        if self.X_scaler == None:
            return np.hstack((spatial_features, hist_features, hog_features))
        else:
            return X_scaler.transform(np.hstack((spatial_features, hist_features, hog_features))) 
        
def get_images():
    car_images = glob.glob("./data/vehicles/**/*.png", recursive=True)
    non_car_images = glob.glob("./data/non-vehicles/**/*.png", recursive=True)
    return car_images, non_car_images

def get_train_test_data(car, non_car, preprocessor):
    car_features = []
    notcar_features = []
    print("Total Images: {}".format(len(car) + len(non_car)))
    i = 0
    for c in car:
        if (i % 1000 == 0):
            print(i)
        preprocessor.set_image(c)
        car_features.append(preprocessor.get_features_for_image())
        i=i+1
    for nc in non_car:
        if (i % 1000 == 0):
            print(i)
        preprocessor.set_image(nc)
        notcar_features.append(preprocessor.get_features_for_image())
        i=i+1

    X = np.vstack((car_features, notcar_features)).astype(np.float64)     
    X_scaler = StandardScaler().fit(X)
    scaled_X = X_scaler.transform(X)
    y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))

    # Split up data into randomized training and test sets
    rand_state = np.random.randint(0, 100)
    X_train, X_test, y_train, y_test = train_test_split(
        scaled_X, y, test_size=0.2, random_state=rand_state)
    return X_train, X_test, y_train, y_test, X_scaler

## Train Model

In [23]:
car, noncar = get_images()
p = PreProcessing()
X_train, X_test, y_train, y_test, X_scaler = get_train_test_data(car, noncar, p)

Total Images: 17760
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
(17760, 7872)


In [25]:
svc = LinearSVC()
svc.fit(X_train, y_train)
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))

Test Accuracy of SVC =  0.9882
